In [7]:
# scripts/convert_yolo_to_cls.py
import os, shutil
from pathlib import Path

def convert_split(split_base, out_base):
    images_path = Path(split_base) / "images"
    labels_path = Path(split_base) / "labels"
    out_train = Path(out_base)
    out_train.mkdir(parents=True, exist_ok=True)

    exts = {".jpg", ".jpeg", ".png"}
    for img_file in images_path.iterdir():
        if img_file.suffix.lower() not in exts: 
            continue
        label_file = labels_path / (img_file.stem + ".txt")
        if not label_file.exists():
            continue
        with open(label_file, "r") as f:
            lines = [l.strip() for l in f.readlines() if l.strip()]
        if not lines:
            continue
        classes = {line.split()[0] for line in lines}
        if len(classes) != 1:
            print(f"Skipping {img_file.name}, multiple classes: {classes}")
            continue
        cls = classes.pop()
        dest = out_train / cls
        dest.mkdir(parents=True, exist_ok=True)
        shutil.copy(img_file, dest / img_file.name)

if __name__ == "__main__":
    base = Path("D:\SEBASTIAN\AOL_AI\condition_classifier\laptop_dataset")
    out_root = Path("D:\SEBASTIAN\AOL_AI\condition_classifier\classification")
    for split in ["train", "valid", "test"]:
        convert_split(base / split, out_root / split)
    print("Conversion done.")


Skipping 12_jpg.rf.5d8bf853de2bf40260ef5dd81631eb26.jpg, multiple classes: {'3', '1'}
Skipping 12_jpg.rf.ac044f70b72ec7804670b0be5542723e.jpg, multiple classes: {'3', '1'}
Skipping 12_jpg.rf.fdb082f5854697de29e8e0ff241e4fb8.jpg, multiple classes: {'3', '1'}
Skipping 13_jpg.rf.11ef2eb98d9a0eb01dc504ebf602c969.jpg, multiple classes: {'1', '4'}
Skipping 13_jpg.rf.59d2a66c7ecfb75f39a443151c3328ef.jpg, multiple classes: {'1', '4'}
Skipping 13_jpg.rf.c9d9b2b2df8bfa1b257803f9eef5f567.jpg, multiple classes: {'1', '4'}
Skipping 23_jpg.rf.042ec3320dd746b89c1ad3fe2d245f45.jpg, multiple classes: {'1', '3', '4'}
Skipping 23_jpg.rf.3d7e70b4f910e345e3e5966e22e5a9b2.jpg, multiple classes: {'1', '3', '4'}
Skipping 23_jpg.rf.4ac573c40709eedb67b9951b34cfbcf6.jpg, multiple classes: {'1', '3', '4'}
Skipping 24_jpg.rf.0bd51d73b7be22f178c23db7411801b5.jpg, multiple classes: {'3', '1'}
Skipping 24_jpg.rf.382ab9100deafdd74f12f08dc9fb7509.jpg, multiple classes: {'3', '1'}
Skipping 24_jpg.rf.c162a2c4bf73ef374f62

In [10]:
import os

def count_files_in_folder(folder_path):
    """
    Counts the number of files in a given folder (excluding subdirectories).

    Args:
        folder_path (str): The path to the folder.

    Returns:
        int: The number of files in the folder.
    """
    count = 0
    # Iterate through all entries in the specified folder
    for entry in os.listdir(folder_path):
        # Construct the full path to the entry
        full_path = os.path.join(folder_path, entry)
        # Check if the entry is a file (and not a directory)
        if os.path.isfile(full_path):
            count += 1
    return count

# Example usage:
directory_to_check = r"D:\SEBASTIAN\AOL_AI\condition_classifier\archive\test\phone_broken"  # Replace with the actual path to your folder

file_count = count_files_in_folder(directory_to_check)
print(f"Number of files in '{directory_to_check}': {file_count}")

Number of files in 'D:\SEBASTIAN\AOL_AI\condition_classifier\archive\test\phone_broken': 11


In [ ]:
# test for dataset phone broken & ok
# phone_good = 12
# phone_broken = 11

# original dataset
# phone_good = 37
# phone_broken = 34

In [3]:
import os
import shutil

# --- PATH CONFIGURATION ---
yolo_root = r"D:\SEBASTIAN\AOL_AI\condition_classifier\phone_dataset"  # Your YOLO dataset
target_root = r"D:\SEBASTIAN\AOL_AI\condition_classifier\classification" # Your Laptop dataset (0-4)

# Map YOLO ID (0,1,2) -> New Folder Name ("5","6","7")
yolo_map = {0: "5", 1: "6", 2: "7"}

def process_yolo(subset):
    # subset is "train", "valid", or "test"
    img_dir = os.path.join(yolo_root, subset, "images")
    lbl_dir = os.path.join(yolo_root, subset, "labels")
    
    if not os.path.exists(lbl_dir): return

    print(f" Processing YOLO {subset}...")
    
    for label_file in os.listdir(lbl_dir):
        if not label_file.endswith(".txt"): continue
        
        # 1. Read Class ID
        with open(os.path.join(lbl_dir, label_file), "r") as f:
            line = f.readline()
            if not line: continue
            class_id = int(line.split()[0])
            
        if class_id not in yolo_map: continue
        target_folder = yolo_map[class_id]

        # 2. Find Image
        base_name = label_file.replace(".txt", "")
        img_name = None
        for ext in [".jpg", ".png", ".jpeg"]:
            if os.path.exists(os.path.join(img_dir, base_name + ext)):
                img_name = base_name + ext
                break
        
        if img_name:
            # 3. Copy to Target
            src = os.path.join(img_dir, img_name)
            dst_dir = os.path.join(target_root, subset, target_folder)
            os.makedirs(dst_dir, exist_ok=True)
            shutil.copy(src, os.path.join(dst_dir, img_name))

# Run for all splits
process_yolo("train")
process_yolo("valid")
process_yolo("test")

 Processing YOLO train...
 Processing YOLO valid...
 Processing YOLO test...


In [4]:
import random

# --- PATH CONFIGURATION ---
archive_root = r"D:\SEBASTIAN\AOL_AI\condition_classifier\archive"
target_root = r"D:\SEBASTIAN\AOL_AI\condition_classifier\classification"

# Map Folder Name -> New Class ID
archive_map = {"brokenphone": "8", "phone": "9"}

def split_archive_data():
    for folder_name, class_id in archive_map.items():
        src_path = os.path.join(archive_root, folder_name)
        if not os.path.exists(src_path): continue
        
        # Get all images
        files = [f for f in os.listdir(src_path) if f.lower().endswith(('.png', '.jpg'))]
        random.shuffle(files) # Shuffle for random split
        
        # Calculate split indices
        total = len(files)
        n_train = int(total * 0.8)
        n_valid = int(total * 0.9) # The 10% between 80% and 90%
        
        splits = {
            "train": files[:n_train],
            "valid": files[n_train:n_valid],
            "test":  files[n_valid:]
        }
        
        print(f"Processing Archive '{folder_name}' -> Class {class_id}")
        
        for split, file_list in splits.items():
            dst_dir = os.path.join(target_root, split, class_id)
            os.makedirs(dst_dir, exist_ok=True)
            
            for fname in file_list:
                shutil.copy(os.path.join(src_path, fname), os.path.join(dst_dir, fname))

split_archive_data()

Processing Archive 'brokenphone' -> Class 8
Processing Archive 'phone' -> Class 9


In [6]:
# --- PATH CONFIGURATION ---
synthetic_root = r"D:\SEBASTIAN\AOL_AI\condition_classifier\filtered_data"
target_root = r"D:\SEBASTIAN\AOL_AI\condition_classifier\classification"

def inject_synthetic():
    print("📦 Injecting Synthetic Data (Train/Valid Split)...")
    
    # Group files by class first
    files_by_class = {"8": [], "9": []}
    
    for fname in os.listdir(synthetic_root):
        if not fname.endswith(".png"): continue
        
        # Check filename prefix
        if fname.startswith("brokenphone"):
            files_by_class["8"].append(fname)
        elif fname.startswith("phone"): # Assuming good phones start with 'phone'
            files_by_class["9"].append(fname)
            
    # Process each class
    for class_id, files in files_by_class.items():
        random.shuffle(files)
        
        split_idx = int(len(files) * 0.8) # 80% Train, 20% Valid
        train_files = files[:split_idx]
        valid_files = files[split_idx:]
        
        # Copy to Train
        dst_train = os.path.join(target_root, "train", class_id)
        os.makedirs(dst_train, exist_ok=True)
        for f in train_files:
            shutil.copy(os.path.join(synthetic_root, f), os.path.join(dst_train, f"syn_{f}"))
            
        # Copy to Valid
        dst_valid = os.path.join(target_root, "valid", class_id)
        os.makedirs(dst_valid, exist_ok=True)
        for f in valid_files:
            shutil.copy(os.path.join(synthetic_root, f), os.path.join(dst_valid, f"syn_{f}"))
            
        print(f"   Class {class_id}: Added {len(train_files)} to Train, {len(valid_files)} to Valid.")

inject_synthetic()


📦 Injecting Synthetic Data (Train/Valid Split)...
   Class 8: Added 92 to Train, 23 to Valid.
   Class 9: Added 109 to Train, 28 to Valid.
